In [32]:
import pandas as pd
# load data
df = pd.read_csv('../data/processed/merged_data.csv')

## Dim and Fact table

### Dim Table

FOE Table

In [36]:
dim_foe = (
  df.loc[:,['foe_code','foe_detailed']]
    .drop_duplicates()
    .rename(columns={'foe_detailed': 'FOE_name'})
    .reset_index(drop=True)
)
dim_foe.head()

,foe_code,FOE_name
0,10100,Mathematical Sciences
1,10101,Mathematics
2,10103,Statistics
3,10199,"Mathematical Sciences, n.e.c."
4,10300,Physics and Astronomy


Fees Table

In [37]:
dim_fees = (
    df.loc[df['funding_type'] != '0', ['funding_type', 'funding_nation']]
    .drop_duplicates()
    .rename(columns={'funding_nation': 'student_type'})
    .reset_index(drop=True)
    .assign(fee_code=lambda x: x.index + 1)
    [['fee_code', 'funding_type', 'student_type']]
)

dim_fees.head(10)

,fee_code,funding_type,student_type
0,1,C'wealth Supported,Domestic
1,2,Fee-Paying,Domestic
2,3,Onshore,International
3,4,Non-Award & Others,Domestic
4,5,Onshore Exchange,International
5,6,Study Abroad,International
6,7,RTP,Domestic
7,8,RTP,International
8,9,Offshore,International


Cluster Table

In [38]:
dim_fundingcluster = (
    df.loc[df['funding_cluster'] != '0', ['funding_cluster']]
    .drop_duplicates()
    .reset_index(drop=True)
    .assign(cluster_code=lambda x: x.index + 1)
    [['cluster_code','funding_cluster']]
)

dim_fundingcluster.head()

,cluster_code,funding_cluster
0,1,Funding Cluster 2
1,2,Funding Cluster 3
2,3,Funding Cluster 4
3,4,Funding Cluster 1


Course Table

In [41]:
dim_course = (
    df.loc[df['course_id'] != 0, ['course_id', 'course_type_broad']]
    .drop_duplicates()
    .reset_index(drop=True)
    .rename(columns={'course_type_broad':'course_type'})
)

dim_course.head()

,course_id,course_type
0,2951.0,Undergraduate
1,2954.0,Undergraduate
2,2976.0,Undergraduate
3,2978.0,Undergraduate
4,3028.0,Undergraduate


Fact Table

In [47]:
df.columns

Index(['course_id', 'course_type_broad', 'unit_id', 'unit_level_code',
       'unit_level_name', 'unit_foe_detailed', 'unit_foe_narrow', 'foe_code',
       'unit_foe_broad', 'eftsl_2024', 'funding_nation', 'funding_type',
       'overload', 'funding_cluster', 'max_student_contrib_2024',
       'commonwealth_contrib_2024', 'max_student_contrib_gf_2024',
       'commonwealth_contrib_gf_2024', 'is_funding_cluster_variable',
       'special_course_code', 'max_contrib_indicator', 'foe_detailed_title',
       'foe_detailed', 'foe_narrow', 'foe_broad', 'foe_error', 'special_code',
       'CSP_gov_payment'],
      dtype='object')

In [49]:
df['funding_type'].value_counts()

funding_type
C'wealth Supported    10528
Onshore                5121
Fee-Paying              733
Onshore Exchange        568
RTP                     328
Study Abroad            237
0                       225
Non-Award & Others      150
Offshore                  3
Name: count, dtype: int64

In [58]:
fact_table = (
    df.loc[
        (df['is_funding_cluster_variable'] != '0') &
        (df['funding_nation'] == 'Domestic') &
        (df['funding_type'] == "C'wealth Supported"),

        ['foe_code', 'eftsl_2024', 'CSP_gov_payment']
    ]
    .reset_index(drop=True)
    .rename(columns={'eftsl_2024': 'eftsl'})
)

len(fact_table['foe_code'])

10162